In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator, FixedOffsetFrame
    )

from panda_station import *

In [2]:
"""
builder = DiagramBuilder()
station = PandaStation()


names_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]
station.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)
station.add_panda_with_hand()
plant, scene_graph = station.get_plant_and_scene_graph()

#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))
station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),
                           main_body_name = "base_link", welded = True, name = "A")
station.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),
                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,
                            name = "B"
                           )


station.finalize()

builder.AddSystem(station)
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(1.0)

print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSE TABLES")
print(parse_tables(find_resource("directives/three_tables.yaml")))

pose_fluents = [
    ("grasppose", "B", RigidTransform(RotationMatrix(), [0, 0, 0.3])),
    ("atpose", "A", RigidTransform(RotationMatrix(), [0.8, 0.2, 0.2]))
]
update_station(station, station_context, pose_fluents)

simulator_context = simulator.get_context()
plant_context = plant.GetMyContextFromRoot(simulator_context)
station_context = station.GetMyContextFromRoot(simulator_context)

simulator.AdvanceTo(2.0)
meshcat.stop_recording()
meshcat.publish_recording()
"""

'\nbuilder = DiagramBuilder()\nstation = PandaStation()\n\n\nnames_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]\nstation.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)\nstation.add_panda_with_hand()\nplant, scene_graph = station.get_plant_and_scene_graph()\n\n#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))\nstation.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),\n                           main_body_name = "base_link", welded = True, name = "A")\nstation.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),\n                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,\n                            name = "B"\n                           )\n\n\ns

In [5]:
p = ProblemInfo("/home/agrobenj/drake-tamp/experiments/pick-place-regions/problems/test_problem.yaml")
stations = p.make_all_stations()
print(list(stations.keys()))
builder = DiagramBuilder()
station = stations["move_free"]
builder.AddSystem(station)
plant, scene_graph = station.get_plant_and_scene_graph()
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()

director = builder.AddSystem(TrajectoryDirector())
builder.Connect(station.GetOutputPort("panda_position_measured"), director.GetInputPort("panda_position"))
builder.Connect(director.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
if (not station.weld_fingers):
    builder.Connect(station.GetOutputPort("hand_state_measured"), director.GetInputPort("hand_state"))
    builder.Connect(director.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)

meshcat.start_recording()
simulator.AdvanceTo(0.2)


update_station(station, station_context, pose_fluents = [])
q1 = np.array([0.0, 0.1, 0, -1.2, 0, 1.6, 0])
q2 = q1.copy()
q2[0] = np.pi/2
panda_traj = np.array([q1, q2])
#find_traj(station, station_context, q1, q2)#
print(panda_traj)
panda_times = np.linspace(0.2, 5.0, len(panda_traj))
print(panda_times)
panda_traj = PiecewisePolynomial.FirstOrderHold(panda_times, panda_traj.T)
qh1 = np.array([0])
qh2 = np.array([0.08])
samples = np.array([qh1, qh2])
t = np.array([0.2, 5.0])
hand_traj = PiecewisePolynomial.FirstOrderHold(t, samples.T)
director.add_panda_traj(panda_traj)
director.add_hand_traj(hand_traj)
simulator.AdvanceTo(5.0)
meshcat.stop_recording()
meshcat.publish_recording()

print(plant.num_positions())
"""
print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSING TABLES")
print(parse_tables(station.directive))
"""

{'foam_brick': {'path': 'models/manipulands/sdf/foam_brick.sdf', 'X_WO': RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [-0.0, 0.0, 1.0],
  ]),
  p=[0.6, 0.0, 0.2],
), 'main_link': 'base_link'}, 'meat_can': {'path': 'models/manipulands/sdf/meat_can.sdf', 'X_WO': RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [-0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, 0.2, 0.2],
), 'main_link': 'base_link_meat'}, 'soup_can': {'path': 'models/manipulands/sdf/soup_can.sdf', 'X_WO': RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [-0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, -0.2, 0.2],
), 'main_link': 'base_link_soup'}}
['main', 'move_free', 'foam_brick', 'meat_can', 'soup_can']
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the fo

'\nprint("PARSING STATION POSES")\nprint(parse_start_poses(station, station_context))\nprint("PARSING INITAL CONFIG")\nprint(parse_config(station, station_context))\nprint("PARSING TABLES")\nprint(parse_tables(station.directive))\n'

In [4]:
plant, scene_graph = station.get_plant_and_scene_graph()
plant_context = station.GetSubsystemContext(plant, station_context)
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
panda = station.get_panda()
hand = station.get_hand()
query_output_port = scene_graph.GetOutputPort("query")

def is_colliding(q):
    plant.SetPositions(plant_context, panda, q)
    query_object = query_output_port.Eval(scene_graph_context)
    pairs = query_object.ComputePointPairPenetration()
    print('pairs:')
    for p in pairs:
        print(p.id_A, p.id_B)
    print("panda hand")
    ids = plant.GetBodyIndices(hand)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    print("panda")
    ids = plant.GetBodyIndices(panda)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    print("brick")
    brick = plant.GetModelInstanceByName("foam_brick")
    ids = plant.GetBodyIndices(brick)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    table = plant.GetModelInstanceByName("table")
    ids = plant.GetBodyIndices(table)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    return query_object.HasCollisions()

def isStateValid(state):
    q = state_to_q(state)
    return not is_colliding(q)

print(is_colliding(q1))

pairs:
panda hand
<GeometryId value=329>
<GeometryId value=332>
<GeometryId value=333>
<GeometryId value=336>
<GeometryId value=337>
panda
<GeometryId value=198>
<GeometryId value=200>
<GeometryId value=202>
<GeometryId value=204>
<GeometryId value=206>
<GeometryId value=208>
<GeometryId value=210>
<GeometryId value=212>
<GeometryId value=214>
<GeometryId value=216>
<GeometryId value=218>
<GeometryId value=220>
<GeometryId value=222>
<GeometryId value=224>
<GeometryId value=228>
<GeometryId value=230>
<GeometryId value=232>
<GeometryId value=234>
<GeometryId value=236>
<GeometryId value=238>
<GeometryId value=242>
<GeometryId value=244>
<GeometryId value=246>
<GeometryId value=248>
<GeometryId value=250>
<GeometryId value=252>
<GeometryId value=256>
<GeometryId value=258>
<GeometryId value=260>
<GeometryId value=262>
<GeometryId value=264>
<GeometryId value=268>
<GeometryId value=270>
<GeometryId value=272>
<GeometryId value=274>
<GeometryId value=276>
<GeometryId value=278>
<GeometryI